In [ ]:
import polars as pl
import networkx as nx
from   pathlib import Path
import json
import rtsvg
rt = rtsvg.RACETrack()
coverage = json.load(open(Path('../../20250921_coverage_json_files/20251022_2010_coverage.json'), 'rt'))
# coverage['files']['rtsvg/circle_packer.py'].keys()
_file_order_ = [_tuple_[1] for _tuple_ in sorted([(int(coverage['files'][_file_]['summary']['percent_covered_display']), _file_) for _file_ in coverage['files'].keys()])]
_tiles_ = []
for _file_ in _file_order_:
    _executed_lines_ = coverage['files'][_file_]['executed_lines']
    _missing_lines_  = coverage['files'][_file_]['missing_lines']
    _excluded_lines_ = coverage['files'][_file_]['excluded_lines']
    _lu_ = {'line':[], 'status':[]}
    for _status_ in ['executed_lines', 'missing_lines', 'excluded_lines']:
        for _line_ in coverage['files'][_file_][_status_]:
            _lu_['line'].append(_line_), _lu_['status'].append(_status_)
    rt.co_mgr.str_to_color_lu['executed_lines'] = '#0000ff' # blue
    rt.co_mgr.str_to_color_lu['missing_lines']  = '#ff0000' # red
    rt.co_mgr.str_to_color_lu['excluded_lines'] = '#ff8000' # orange
    _xy_ = rt.xy(pl.DataFrame(_lu_), x_field='line', y_field='status', color_by='status', dot_size='medium', w=512, h=64, y_ins=10)
    _perc_covered_ = coverage['files'][_file_]['summary']['percent_covered_display']
    _tiles_.append(rt.titleSVG(_xy_, f'{_file_} {_perc_covered_}%'))
#rt.table(_tiles_, per_row=3, spacer=10)

In [ ]:
#import inspect
#dir(rt)
#inspect.getmembers(rt)

In [ ]:
#help(inspect)

In [ ]:
#inspect.getsourcelines(rtsvg)

In [ ]:
#
# Trying to implement the graph from the following paper:
#
# C. Collberg, S. G. Kobourov, J. Nagra, J. Pitts, and K. Wampler. 
# A system for graph-based visualization of the evolution of software. 
# In ACM Symposium on Software Visualization (SoftVis), pages 77–86, 2003.
#
# FunctionCallCollector prototype from Google AI
#
import ast
import os
class FunctionCallCollector(ast.NodeVisitor):
    def __init__(self, file):
        self.file                 = file
        self.hierarchy            = []
        self.graph_edges          = {'fm':[], 'fm_type':[], 'to':[], 'to_type':[]}
        self.location_info        = {'hierarchy':[], 'type':[], 'lineno':[], 'file':[]}
    def __recordLocation__(self):
        self.location_info['hierarchy'].append(self.hierarchyString(include_last=True))
        self.location_info['type']     .append(self.hierarchy[-1][1])
        self.location_info['lineno']   .append(self.hierarchy[-1][2])
        self.location_info['file']     .append(self.file)
        pass
    def hierarchyString(self, include_last=True):
        strs   = []
        offset = 0 if include_last else 1
        for i in range(len(self.hierarchy)-offset): strs.append(self.hierarchy[i][0])
        return '.'.join(strs)
    def visit_FunctionDef(self, node):
        self.hierarchy.append((node.name, 'function', node.lineno))
        self.__recordLocation__()
        if len(self.hierarchy) >= 2:
            self.graph_edges['fm'].append(self.hierarchyString(False)), self.graph_edges['fm_type'].append(self.hierarchy[-2][1])
            self.graph_edges['to'].append(self.hierarchyString()),      self.graph_edges['to_type'].append(self.hierarchy[-1][1])
        self.generic_visit(node)
        self.hierarchy.pop()
    def visit_Call(self, node):
        _name_ = None
        if   isinstance(node.func, ast.Name):      _name_ = node.func.id
        elif isinstance(node.func, ast.Attribute): _name_ = node.func.attr
        elif isinstance(node.func, ast.Call):      _name_ = node.func.func.id
        elif isinstance(node.func, ast.Subscript): _name_ = '[]' # i'm guessing...
        else:                                      print(f'!!! {type(node.func)}')
        if len(self.hierarchy) > 0:
            self.graph_edges['fm'].append(self.hierarchyString()), self.graph_edges['fm_type'].append(self.hierarchy[-1][1])
            self.graph_edges['to'].append(_name_),                 self.graph_edges['to_type'].append('call')
        self.generic_visit(node)
    def visit_ClassDef(self, node):
        self.hierarchy.append((node.name, 'class', node.lineno))
        self.__recordLocation__()
        self.generic_visit(node)
        self.hierarchy.pop()
_file_to_lines_ = {}
ast_parses      = []
_dfs_           = []
_dfs_location_  = []
for _file_ in os.listdir('../rtsvg'):
    if _file_.endswith('.py'):
        with open(os.path.join('../rtsvg', _file_), 'rt') as _f_:
            _file_to_lines_[_file_] = _f_.readlines()
        _ast_parse_ = ast.parse(''.join(_file_to_lines_[_file_]))
        ast_parses.append(_ast_parse_)
        collector = FunctionCallCollector(_file_)
        collector.visit(_ast_parse_)
        _dfs_.append(pl.DataFrame(collector.graph_edges))
        _dfs_location_.append(pl.DataFrame(collector.location_info))
df          = pl.concat(_dfs_).drop_nulls()
df_location = pl.concat(_dfs_location_)
g   = rt.createNetworkXGraph(df, [('fm','to')])
_params_ = {'relationships':[('fm','to')]}
_igp_    = rt.interactiveGraphPanel(df, ln_params=_params_, w=800, h=700)
#_igp_

In [ ]:
import pyclbr
_result_ = pyclbr.readmodule('linknode_graph_patterns')
_result_

In [ ]:
rt.tile([rt.histogram(df_location, bin_by='type', color_by='file', w=400, h=700),
         rt.histogram(df_location, bin_by='file', color_by='file', w=400, h=700)])

In [ ]:
_dfs_ = []
for _file_, _df_ in df_location.group_by('file'):
    _df_ = _df_.sort(by='lineno')
    _df_ = _df_.with_columns(pl.col('lineno').shift(-1).alias('lineno_last'))
    _dfs_.append(_df_)
df_location_wel = pl.concat(_dfs_) # _wel == With End Line
df_location_wel.sort(by=['file', 'lineno'])
